In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
from pathlib import Path
import itertools
import functools
from collections import namedtuple, Counter, defaultdict
import re
import pickle
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import read_vocab, PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID, IMAGE_H, IMAGE_W
from multimodal.multimodal_saycam_data_module import VOCAB_FILENAME
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import *
from analysis_tools.utils import *
from analysis_tools.sentences_data import *
from analysis_tools.sentences_utils import *
from analysis_tools.checkpoints import *
from analysis_tools.build_data import *
import analysis_tools.plotting as plotting
import textwrap

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']


ZORRO_TXT_DIR = Path("/misc/vlgscratch4/LakeGroup/shared_data/Zorro/sentences/babyberta/")
ZORRO_TXT_FILES = ZORRO_TXT_DIR.glob('*.txt')
TRAIN_MEAN_IMG_FILE = Path("/misc/vlgscratch4/LakeGroup/shared_data/S_multimodal/train_mean_img.pt")


# set default settings for plotting; may change for each plot
figsize = (12, 12)
paper_context = sns.plotting_context('paper')
paper_context.update({
    'font.size': 10.,
    'axes.labelsize': 10.,
    'axes.titlesize': 14.,
    'xtick.labelsize': 8.8,
    'ytick.labelsize': 8.8,
    'legend.fontsize': 8.8,
    'legend.title_fontsize': 9.6,
})
unticked_relation_style = sns.axes_style('white')
unticked_relation_style.update({
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.spines.right': False,
    'axes.spines.top': False,
})
font = 'serif'
sns.set_theme(
    #context=paper_context,
    style=unticked_relation_style,
    #palette=sns.color_palette('tab20'),
    font='serif',
    rc={
        'figure.figsize': figsize,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams["savefig.bbox"] = 'tight'
plt.rcParams["savefig.pad_inches"] = 0.
plot_format = 'pdf'
saving_fig = True

if saving_fig:
    def _save_fig(fname, format='png'):
        print(f'saving plot {fname}')
        plt.savefig(f'plots/{fname}.{format}', transparent=True)
        plt.clf()
    plotting.output_fig = functools.partial(_save_fig, format=plot_format)
else:
    plotting.output_fig = lambda fname: plt.show()

output_fig = plotting.output_fig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def sentence_pair_in_vocab(sentence_pair, vocab):
    for sentence in sentence_pair:
        if not all(str(token) in vocab for token in sentence):
            return False
    return True


phenomenon_name_aliases = {
    'agreement_determiner_noun': 'determiner-noun_agreement',
    'agreement_subject_verb'   : 'subject-verb_agreement'   ,
}


def get_data_name_from_paradigm_name(
    paradigm_name,
    sep='-',
    replace_underline_by=' ',
    phenomenon_name_aliases=phenomenon_name_aliases,
):
    partition_of_paradigm_name = paradigm_name.rpartition(sep)
    assert partition_of_paradigm_name[1] == sep
    phenomenon_name, _paradigm_name = partition_of_paradigm_name[0], partition_of_paradigm_name[2]
    phenomenon_name = phenomenon_name_aliases.get(phenomenon_name, phenomenon_name)
    data_name = phenomenon_name, _paradigm_name
    if replace_underline_by is not None:
        data_name = tuple(name.replace('_', replace_underline_by) for name in data_name)
    return data_name

/tmp/ipykernel_407499/2621710530.py:31: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']


In [3]:
# get SAYCam vocab
saycam_vocab = read_vocab(VOCAB_FILENAME)
saycam_vocab_size = len(saycam_vocab)
print(f'{saycam_vocab_size = }')
word2idx = saycam_vocab
idx2word = [None] * saycam_vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

# get Zorro data
zorro_data = {}
zorro_vocab = Counter()

for zorro_txt_file in ZORRO_TXT_FILES:
    paradigm_name = zorro_txt_file.stem
    data_name = get_data_name_from_paradigm_name(paradigm_name)
    print(f'processing {paradigm_name} ...', flush=True)

    with open(zorro_txt_file, 'r') as f:
        sentences = pd.Series(tokenize(line.strip() for line in f))

    zorro_data[data_name] = sentences

    zorro_vocab.update(map(str, itertools.chain.from_iterable(sentences)))

# Zorro vocab
zorro_vocab = sorted(
    zorro_vocab.items(),
    key = lambda item: (-item[1], item[0])
)
zorro_vocab_size = len(zorro_vocab)
print(f'{zorro_vocab_size = }')
print('words in zorro vocab:')
for word, freq in zorro_vocab[:100]:
    print(f'{word:10} {freq:6}')

# Zorro vocab compared to SAYCam vocab
in_saycam_vocab_cnt = 0
print('words not in SAYCam vocab:')
for word, freq in zorro_vocab:
    if word in saycam_vocab:
        in_saycam_vocab_cnt += 1
    else:
        print(word)
print(f'{frac_format(in_saycam_vocab_cnt, len(zorro_vocab))} in SAYCam vocab')

# filter Zorro data
filter_masks = {}

for data_name, sentences in zorro_data.items():
    n_total = 0

    print('out-of-SAYCam vocab sentence examples:')
    sentence_pairs = pd.Series(paired(sentences))
    filter_mask = sentence_pairs.map(lambda sentence_pair: sentence_pair_in_vocab(sentence_pair, saycam_vocab))
    for sentence_pair in sentence_pairs[~filter_mask].iloc[:0]:
        for sentence in sentence_pair:
            print(untokenize(sentence))
        print(
            'OOV tokens:',
            ' '.join(
                str(token)
                for sentence in sentence_pair
                for token in sentence
                if str(token) not in saycam_vocab
            )
        )

    print(f'{data_name}: {frac_format(filter_mask.sum(), len(filter_mask))}')

    filter_masks[data_name] = filter_mask

saycam_vocab_size = 2350
processing agreement_determiner_noun-across_1_adjective ...
processing agreement_determiner_noun-between_neighbors ...
processing agreement_subject_verb-across_prepositional_phrase ...
processing agreement_subject_verb-across_relative_clause ...
processing agreement_subject_verb-in_question_with_aux ...
processing agreement_subject_verb-in_simple_question ...
processing anaphor_agreement-pronoun_gender ...
processing argument_structure-dropped_argument ...
processing argument_structure-swapped_arguments ...
processing argument_structure-transitive ...
processing binding-principle_a ...
processing case-subjective_pronoun ...
processing ellipsis-n_bar ...
processing filler-gap-wh_question_object ...
processing filler-gap-wh_question_subject ...
processing irregular-verb ...
processing island-effects-adjunct_island ...
processing island-effects-coordinate_structure_constraint ...
processing local_attractor-in_question_with_aux ...
processing npi_licensing-matrix_q

In [4]:
# build datasets
datasets = {
    data_name: SentencesDataset(sentences, saycam_vocab)
    for data_name, sentences in zorro_data.items()
}

my_batch_size = 256


# load model from checkpoint

# select from list of checkpoints
dataset_name = "saycam"
names = [
    "LSTM",
    "LSTM Captioning",
    "CBOW",
    "unigram", "bigram", #"trigram", "4-gram",
]
groups = {name: all_groups[name] for name in names}
ori_names = list(itertools.chain.from_iterable(groups.values()))
checkpoint_paths = all_checkpoint_paths[dataset_name]

ori_models = {}

saycam_data = None

for ori_name in ori_names:
    checkpoint_path = checkpoint_paths[ori_name]

    if "gram" in checkpoint_path:
        ngram_model = build_ngram_model(int(checkpoint_path.split('-')[0]), saycam_vocab_size, saycam_data.train_dataloader())
        model = ngram_model

    else:
        print(f"load model from {checkpoint_path}")
        lit_model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
        lit_model.to(device)
        lit_model.eval()
        model = lit_model

        if saycam_data is None:
            # build data and vocab according to the model
            saycam_data, args = build_data(args=lit_model.args, return_args=True)
            dataset_name = args.dataset

        else:
            assert lit_model.args["dataset"] == dataset_name, f"checkpoint {checkpoint_path} ran on a different dataset {args.dataset}"

    ori_models[ori_name] = model

load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt


/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Shuffled training metadata file has already been created. Skipping this step.
Evaluation frames have already been filtered. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Filtered evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipp

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2108/2108 [00:27<00:00, 75.62it/s]


In [5]:
if TRAIN_MEAN_IMG_FILE.exists():
    train_mean_img = torch.load(TRAIN_MEAN_IMG_FILE)
else:
    train_dataset = saycam_data.datasets['train']
    train_mean_img = get_data_mean_img(train_dataset.data)
    torch.save(train_mean_img, TRAIN_MEAN_IMG_FILE)

from multimodal.attention_maps import n_inv
plt.imshow(n_inv(train_mean_img).permute((1, 2, 0)).numpy())
plt.axis('off')
output_fig('Train Mean Image')

train_mean_img = train_mean_img.to(device)


def prepend_image_placeholder_for_batch(batch, zeros=False):
    batch_size = len(batch[0])
    if zeros:
        x = torch.zeros(batch_size, 3, IMAGE_H, IMAGE_W, dtype=torch.float, device=device)
    else:
        x = train_mean_img.unsqueeze(0)
        x = x.expand(*([batch_size] + [-1] * train_mean_img.dim()))
    return x, *batch

saving plot Train Mean Image


<Figure size 3600x3600 with 0 Axes>

In [6]:
loss_dfs = {
    data_name: pd.DataFrame(np.zeros((len(dataset), 0)))
    for data_name, dataset in datasets.items()
}


saycam_model_format = 'SAYCam {}'.format

my_groups = {saycam_model_format(name): list(map(saycam_model_format, group)) for name, group in groups.items()}

for model_name, model in ori_models.items():
    print(f'running {model_name} ...')
    saycam_model_name = saycam_model_format(model_name)
    for data_name, dataset in datasets.items():
        if not dataset:
            continue
        dataloader = DataLoader(
            dataset,
            collate_fn=collate_fn,
            shuffle=False,
            batch_size=my_batch_size,
            drop_last=False,
        )
        batches = map(prepend_image_placeholder_for_batch, dataloader)
        losses = get_model_losses_on_data_batches(model, batches, use_tqdm=True)
        losses = losses.cpu().numpy()
        loss_dfs[data_name][saycam_model_name] = losses

for probing_results_dir in Path('/misc/vlgscratch4/LakeGroup/wentao/BabyBERTa/probing_results/babyberta').glob('*'):
    model_name = probing_results_dir.name
    if model_name not in ['BabyBERTa_AO-CHILDES', 'BabyBERTa_AO-CHILDES+AO-Newselsa+Wikipedia-1'][:1]:
        continue
    name_, _, dataset_ = model_name.partition('_')
    model_name = dataset_ + ' ' + 'Transformer'
    my_groups[model_name] = [model_name]
    for file in probing_results_dir.glob('*'):
        paradigm_name = file.stem
        data_name = get_data_name_from_paradigm_name(paradigm_name)
        sentences, losses = zip(*read_sentences_and_losses(file))
        loss_dfs[data_name][model_name] = losses

running LSTM 0 ...


16it [00:00, 130.64it/s]
16it [00:00, 149.04it/s]
16it [00:00, 135.61it/s]
16it [00:00, 136.96it/s]
16it [00:00, 143.04it/s]
16it [00:00, 146.24it/s]
16it [00:00, 135.47it/s]
16it [00:00, 149.63it/s]
16it [00:00, 142.46it/s]
16it [00:00, 162.56it/s]
16it [00:00, 130.95it/s]
16it [00:00, 147.34it/s]
16it [00:00, 123.24it/s]
16it [00:00, 130.93it/s]
16it [00:00, 131.20it/s]
16it [00:00, 135.81it/s]
16it [00:00, 131.78it/s]
16it [00:00, 139.69it/s]
16it [00:00, 141.58it/s]
16it [00:00, 143.28it/s]
16it [00:00, 136.18it/s]
16it [00:00, 137.95it/s]
16it [00:00, 123.87it/s]


running LSTM 1 ...


16it [00:00, 162.26it/s]
16it [00:00, 165.76it/s]
16it [00:00, 149.71it/s]
16it [00:00, 149.97it/s]
16it [00:00, 155.41it/s]
16it [00:00, 158.66it/s]
16it [00:00, 144.90it/s]
16it [00:00, 154.64it/s]
16it [00:00, 154.51it/s]
16it [00:00, 178.33it/s]
16it [00:00, 135.01it/s]
16it [00:00, 147.47it/s]
16it [00:00, 121.61it/s]
16it [00:00, 132.87it/s]
16it [00:00, 131.86it/s]
16it [00:00, 133.18it/s]
16it [00:00, 132.69it/s]
16it [00:00, 139.78it/s]
16it [00:00, 142.87it/s]
16it [00:00, 143.06it/s]
16it [00:00, 138.97it/s]
16it [00:00, 138.45it/s]
16it [00:00, 126.57it/s]


running LSTM 2 ...


...

16it [00:00, 244.58it/s]
16it [00:00, 251.03it/s]
16it [00:00, 234.93it/s]
16it [00:00, 237.33it/s]
16it [00:00, 239.78it/s]
16it [00:00, 253.26it/s]
16it [00:00, 240.63it/s]
16it [00:00, 256.63it/s]
16it [00:00, 244.27it/s]
16it [00:00, 276.23it/s]
16it [00:00, 223.38it/s]
16it [00:00, 246.76it/s]
16it [00:00, 212.80it/s]
16it [00:00, 223.98it/s]
16it [00:00, 221.74it/s]
16it [00:00, 229.50it/s]
16it [00:00, 221.66it/s]
16it [00:00, 230.22it/s]
16it [00:00, 239.28it/s]
16it [00:00, 239.06it/s]
16it [00:00, 232.73it/s]
16it [00:00, 228.94it/s]
16it [00:00, 214.13it/s]


running 1-gram ...


16it [00:01, 14.64it/s]
16it [00:01, 14.84it/s]
16it [00:01, 14.14it/s]
16it [00:01, 14.22it/s]
16it [00:01, 14.52it/s]
16it [00:01, 14.79it/s]
16it [00:01, 14.42it/s]
16it [00:01, 14.86it/s]
16it [00:01, 14.50it/s]
16it [00:01, 15.54it/s]
16it [00:01, 13.88it/s]
16it [00:01, 14.47it/s]
16it [00:01, 13.46it/s]
16it [00:01, 13.81it/s]
16it [00:01, 13.81it/s]
16it [00:01, 14.13it/s]
16it [00:01, 13.83it/s]
16it [00:01, 14.17it/s]
16it [00:01, 14.37it/s]
16it [00:01, 14.39it/s]
16it [00:01, 14.16it/s]
16it [00:01, 13.94it/s]
16it [00:01, 13.52it/s]


running 2-gram ...


16it [00:01, 14.49it/s]
16it [00:01, 14.53it/s]
16it [00:01, 13.99it/s]
16it [00:01, 14.06it/s]
16it [00:01, 14.41it/s]
16it [00:01, 14.44it/s]
16it [00:01, 14.28it/s]
16it [00:01, 14.69it/s]
16it [00:01, 14.37it/s]
16it [00:01, 15.33it/s]
16it [00:01, 13.58it/s]
16it [00:01, 14.22it/s]
16it [00:01, 13.05it/s]
16it [00:01, 13.64it/s]
16it [00:01, 13.71it/s]
16it [00:01, 13.98it/s]
16it [00:01, 13.71it/s]
16it [00:01, 13.91it/s]
16it [00:01, 14.21it/s]
16it [00:01, 14.28it/s]
16it [00:01, 14.03it/s]
16it [00:01, 13.92it/s]
16it [00:01, 13.36it/s]


In [7]:
def get_common_substr_len(strs, index_fn=lambda l: l):
    l = 0
    l_last_space = -1
    while True:
        try:
            c = None
            for s in strs:
                cur_c = s[index_fn(l)]
                if c is None:
                    c = cur_c
                assert c == cur_c
        except (IndexError, AssertionError):
            break
        if c == ' ':
            l_last_space = l
        l += 1
    return l_last_space + 1


def get_sentence_pair_str(sentence_pair):
    strs = list(map(str, sentence_pair))
    prefix_len = get_common_substr_len(strs)
    suffix_len = get_common_substr_len(strs, lambda l: -l-1)
    return strs[0][:prefix_len] + '(' + '/'.join(reversed([s[prefix_len:-suffix_len] for s in strs])) + ')' + strs[0][-suffix_len:]


def get_sentence_pair_example(sentence_pairs):
    return min(map(get_sentence_pair_str, sentence_pairs), key=len)


data_examples = {}
for data_name, sentences in zorro_data.items():
    sentence_pairs = pd.Series(paired(sentences))[filter_masks[data_name]]
    if len(sentence_pairs):
        example_str = get_sentence_pair_example(sentence_pairs)

        # remove the space before the last punctuation
        if example_str[-2] == ' ':
            example_str = example_str[:-2] + example_str[-1:]

        data_examples[data_name] = example_str

phenomenon_examples = defaultdict(list)
for data_name, example_str in data_examples.items():
    phenomenon_examples[data_name[0]].append(example_str)
phenomenon_examples = {phenomenon: min(examples, key=len) for phenomenon, examples in phenomenon_examples.items()}

In [8]:
fontsize = 8
context = {
    'font.size': fontsize,
    'axes.labelsize': fontsize,
    'axes.titlesize': fontsize,
    'xtick.labelsize': fontsize,
    'ytick.labelsize': fontsize,
    'legend.fontsize': fontsize,
    'legend.title_fontsize': fontsize,
}
width = 6.0
height_each = 0.65
height_base = 0.5


random_accuracy = 0.5
sep = '\n'


def get_correct_df(loss_df):
    sub_loss_dfs = [loss_df.iloc[i::2].reset_index(drop=True) for i in range(2)]
    return sub_loss_dfs[0] > sub_loss_dfs[1]


correct_dfs = {
    data_name: get_correct_df(loss_df)
    for data_name, loss_df in loss_dfs.items()
}

filtered_correct_dfs = {
    data_name: correct_df[filter_masks[data_name]]
    for data_name, correct_df in correct_dfs.items()
}


for current_correct_dfs in [correct_dfs, filtered_correct_dfs][1:]:
    data_names = list(current_correct_dfs.keys())

    ori_example_df = pd.concat(
        [current_correct_dfs[data_name] for data_name in data_names],
        keys=data_names,
        names=['phenomenon', 'paradigm', 'idx'],
    )
    example_df = pd.DataFrame({name: ori_example_df[group].mean(axis=1) for name, group in my_groups.items()})
    phenomenon_agg_name = 'mean over subsets'
    phenomenon_order = [
        phenomenon_agg_name,
        'determiner-noun agreement',
        'subject-verb agreement',
        'argument structure',
        'case',
        'local attractor',
        'npi licensing',
        'quantifiers',
    ]
    model_order = [
        'SAYCam unigram', 'SAYCam bigram',
        'SAYCam CBOW',
        'SAYCam LSTM',
        'SAYCam LSTM Captioning',
        'AO-CHILDES Transformer',
    ]

    example_long_df = example_df.reset_index().melt(
        id_vars=['phenomenon', 'paradigm', 'idx'], var_name='model', value_name='accuracy')
    example_long_df['phenomenon paradigm'] = example_long_df['phenomenon'] + sep + example_long_df['paradigm']

    phenomenon_paradigm_df = example_df.groupby(level=['phenomenon', 'paradigm']).mean()
    display(phenomenon_paradigm_df)
    phenomenon_paradigm_long_df = phenomenon_paradigm_df.reset_index().melt(
        id_vars=['phenomenon', 'paradigm'], var_name='model', value_name='accuracy')

    phenomenon_df = phenomenon_paradigm_df.groupby(by=['phenomenon']).mean()
    display(phenomenon_df)

    df = phenomenon_paradigm_df.mean(axis=0)
    display(df)

    with plt.rc_context(context):
        display(example_long_df)
        height = height_base + height_each * len(example_long_df['phenomenon paradigm'].unique())
        g = sns.catplot(
            kind="bar", data=example_long_df, x='accuracy', y='phenomenon paradigm', hue='model',
            hue_order=model_order, errwidth=1,
            height=height, aspect=width/height,
        )
        #g.figure.set_size_inches(7, 8)
        g.ax.set_xlim(right=1)
        g.refline(x=random_accuracy, linestyle='dashed', linewidth=1, color='black')
        output_fig('Syntactic Probing Accuracy on Paradigms')

        phenomenon_paradigm_long_df_ = phenomenon_paradigm_long_df.copy()
        phenomenon_paradigm_long_df_['phenomenon'] = phenomenon_agg_name
        phenomenon_paradigm_long_df_ = pd.concat(
            [phenomenon_paradigm_long_df.copy(), phenomenon_paradigm_long_df_],
            ignore_index=True,
        )
        def _bf(text):
            return ' '.join((r'\textbf{' + piece + r'}' for piece in text.split(' ')))
        def _get_phenomenon_label(phenomenon):
            label = _bf(phenomenon)
            if phenomenon != phenomenon_agg_name:
                label += '\n' + textwrap.fill(phenomenon_examples[phenomenon], width=32)
            return label
        phenomenon_paradigm_long_df_['phenomenon'] = phenomenon_paradigm_long_df_['phenomenon'].map(_get_phenomenon_label)
        phenomenon_order_ = list(map(_get_phenomenon_label, phenomenon_order))
        height = height_base + height_each * len(phenomenon_paradigm_long_df_['phenomenon'].unique())
        g = sns.catplot(
            kind="bar", data=phenomenon_paradigm_long_df_, x='accuracy', y='phenomenon', hue='model',
            order=phenomenon_order_, hue_order=model_order, errorbar=None,
            height=height, aspect=width/height,
        )
        g.ax.set_xlim(right=1)
        g.refline(x=random_accuracy, linestyle='dashed', linewidth=1, color='black')
        output_fig('Syntactic Probing Accuracy Mean on Phenomena Over Paradigms')

        height = height_base + 1.5 * height_each
        g = sns.catplot(
            kind="bar", data=phenomenon_paradigm_long_df, x='accuracy', y='model',
            order=model_order, errorbar=None,
            height=height, aspect=width/height,
        )
        g.ax.set_xlim(right=1)
        g.refline(x=random_accuracy, linestyle='dashed', linewidth=1, color='black')
        output_fig('Syntactic Probing Accuracy Mean Over Paradigms')

    target_model = "SAYCam LSTM"
    ori_target_model = "SAYCam LSTM 0"
    baseline_models = ["SAYCam unigram", "SAYCam bigram", "SAYCam CBOW"]
    ori_baseline_models = ["SAYCam 1-gram", "SAYCam 2-gram", "SAYCam CBOW 0"]
    displayed_models = [ori_target_model] + ori_baseline_models

    for data_name in phenomenon_paradigm_df.index:
        model_mean_series = phenomenon_paradigm_df.loc[data_name]
        target_model_accuracy = model_mean_series[target_model]
        baseline_accuracy = model_mean_series[baseline_models].max()
        if target_model_accuracy >= 0.52 and target_model_accuracy > baseline_accuracy + 0.03:
            sentence_pairs = pd.Series(paired(zorro_data[data_name]))
            sentence_pairs = sentence_pairs[filter_masks[data_name]]
            correct_df = current_correct_dfs[data_name]
            print(f'comparing {data_name} ({frac_format(len(sentence_pairs), len(zorro_data[data_name]) / 2)}):')
            display(model_mean_series)
            for sentence_pair, (idx, corrects) in zip(sentence_pairs, correct_df.iterrows()):
                if corrects[ori_target_model] and not all(corrects[ori_baseline_models]):
                    for sentence in sentence_pair:
                        print(' '.join(map(str, sentence)))
                    display(corrects[displayed_models])

SAYCam LSTM  SAYCam LSTM Captioning  SAYCam CBOW  \
phenomenon                paradigm                                                                        
argument structure        dropped argument                0.778104                0.732160     0.671554   
                          swapped arguments               0.985507                0.950221     0.998110   
                          transitive                      0.630208                0.626736     0.616319   
case                      subjective pronoun              0.946237                0.876028     0.994307   
determiner-noun agreement across 1 adjective              0.648882                0.649898     0.410061   
                          between neighbors               0.705628                0.711580     0.812771   
local attractor           in question with aux            0.808333                0.792361     0.630556   
npi licensing             matrix question                 0.512478                0.581105     0.336898   
                          only npi licensor               0.931707                0.876423     0.804878   
quantifiers               existential there               0.966851                0.979742     0.736648   
                          superlative                     0.544326                0.780142     0.774823   
subject-verb agreement    across prepositional phrase     0.529167                0.525000     0.500000   
                          across relative clause          0.559524                0.536341     0.500000   
                          in question with aux            0.565476                0.560714     0.480952   
                          in simple question              0.614434                0.605662     0.424242   

                                                       SAYCam unigram  SAYCam bigram  AO-CHILDES Transformer  
phenomenon                paradigm                                                                            
argument structure        dropped argument                   0.419355       0.686217                0.695015  
                          swapped arguments                  0.111531       0.982987                0.967864  
                          transitive                         0.502604       0.598958                0.705729  
case                      subjective pronoun                 0.079696       1.000000                0.960152  
determiner-noun agreement across 1 adjective                 0.500000       0.500000                0.963415  
                          between neighbors                  0.500000       0.662338                0.980519  
local attractor           in question with aux               0.877083       0.870833                0.968750  
npi licensing             matrix question                    0.005348       0.628342                0.711230  
                          only npi licensor                  1.000000       0.829268                0.570732  
quantifiers               existential there                  0.839779       0.900552                0.994475  
                          superlative                        1.000000       0.648936                0.957447  
subject-verb agreement    across prepositional phrase        0.500000       0.475000                0.756250  
                          across relative clause             0.500000       0.500000                0.812030  
                          in question with aux               0.500000       0.482143                0.907143  
                          in simple question                 0.500000       0.492823                0.964115

,SAYCam LSTM,SAYCam LSTM Captioning,SAYCam CBOW,SAYCam unigram,SAYCam bigram,AO-CHILDES Transformer
phenomenon,,,,,,
argument structure,0.797940,0.769706,0.761994,0.344497,0.756054,0.789536
case,0.946237,0.876028,0.994307,0.079696,1.000000,0.960152
determiner-noun agreement,0.677255,0.680739,0.611416,0.500000,0.581169,0.971967
local attractor,0.808333,0.792361,0.630556,0.877083,0.870833,0.968750
npi licensing,0.722093,0.728764,0.570888,0.502674,0.728805,0.640981
quantifiers,0.755589,0.879942,0.755735,0.919890,0.774744,0.975961
subject-verb agreement,0.567150,0.556929,0.476299,0.500000,0.487491,0.859884


SAYCam LSTM               0.715124
SAYCam LSTM Captioning    0.718941
SAYCam CBOW               0.646141
SAYCam unigram            0.522360
SAYCam bigram             0.683893
AO-CHILDES Transformer    0.860991
dtype: float64

,phenomenon,paradigm,idx,model,accuracy,phenomenon paradigm
0,determiner-noun agreement,across 1 adjective,0,SAYCam LSTM,0.000000,determiner-noun agreement\nacross 1 adjective
1,determiner-noun agreement,across 1 adjective,1,SAYCam LSTM,0.333333,determiner-noun agreement\nacross 1 adjective
2,determiner-noun agreement,across 1 adjective,2,SAYCam LSTM,0.666667,determiner-noun agreement\nacross 1 adjective
3,determiner-noun agreement,across 1 adjective,3,SAYCam LSTM,0.666667,determiner-noun agreement\nacross 1 adjective
4,determiner-noun agreement,across 1 adjective,4,SAYCam LSTM,0.333333,determiner-noun agreement\nacross 1 adjective
...,...,...,...,...,...,...
39649,quantifiers,superlative,1972,AO-CHILDES Transformer,1.000000,quantifiers\nsuperlative
39650,quantifiers,superlative,1974,AO-CHILDES Transformer,1.000000,quantifiers\nsuperlative
39651,quantifiers,superlative,1980,AO-CHILDES Transformer,1.000000,quantifiers\nsuperlative
39652,quantifiers,superlative,1981,AO-CHILDES Transformer,1.000000,quantifiers\nsuperlative


/home/ww2135/.local/lib/python3.9/site-packages/seaborn/axisgrid.py:745: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  plot_args = [v for k, v in plot_data.iteritems()]


...

was the cars not fixed ?
was the car not fixed ?


SAYCam LSTM 0     True
SAYCam 1-gram     True
SAYCam 2-gram     True
SAYCam CBOW 0    False
Name: 1977, dtype: bool

<Figure size 2355.55x3075 with 0 Axes>

<Figure size 2355.55x1710 with 0 Axes>

<Figure size 1800x442.5 with 0 Axes>